Stage 10b: Time Series/Classification Modeling - Project ImplementationPortfolio Risk Management SystemThis script implements time series and classification modeling with lag featuresand sklearn Pipeline for automated model building.

In [ ]:
import sysimport ossys.path.append('../src')import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom scipy import statsimport utilsfrom sklearn.ensemble import RandomForestClassifier, RandomForestRegressorfrom sklearn.linear_model import LogisticRegressionfrom sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_scorefrom sklearn.preprocessing import StandardScalerfrom sklearn.pipeline import Pipelinefrom sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_scorefrom sklearn.metrics import mean_squared_error, r2_scoreimport warningswarnings.filterwarnings('ignore')print("⏰ Stage 10b: Time Series/Classification Modeling - Portfolio Risk Management")

In [ ]:
def load_and_prepare_data():    """Load data and create time series features"""    print("Loading data for time series modeling...")        symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']    raw_data = utils.fetch_multiple_stocks(symbols, prefer_alphavantage=False, period='2y')        if raw_data.empty:        print("❌ Failed to load data")        return None    

## Create comprehensive time series features

In [ ]:
    processed_data = []    for symbol in symbols:        symbol_data = raw_data[raw_data['symbol'] == symbol].copy().sort_values('date')        

## Base features

In [ ]:
        symbol_data['daily_return'] = symbol_data['close'].pct_change()        symbol_data['log_return'] = np.log(symbol_data['close'] / symbol_data['close'].shift(1))        symbol_data['price_range'] = (symbol_data['high'] - symbol_data['low']) / symbol_data['close']        

## Technical indicators

In [ ]:
        symbol_data['rsi'] = calculate_rsi(symbol_data['close'])        symbol_data['macd'], symbol_data['macd_signal'] = calculate_macd(symbol_data['close'])        symbol_data['bb_position'] = calculate_bollinger_position(symbol_data['close'])        

## Lag features - key for time series

In [ ]:
        lag_vars = ['daily_return', 'log_return', 'price_range', 'rsi']        for var in lag_vars:            for lag in [1, 2, 3, 5]:                symbol_data[f'{var}_lag_{lag}'] = symbol_data[var].shift(lag)        

## Rolling features

In [ ]:
        for window in [5, 10, 20]:            symbol_data[f'return_mean_{window}'] = symbol_data['daily_return'].rolling(window).mean()            symbol_data[f'return_std_{window}'] = symbol_data['daily_return'].rolling(window).std()            symbol_data[f'volume_mean_{window}'] = symbol_data['volume'].rolling(window).mean()        

## Volume features

In [ ]:
        symbol_data['volume_ratio'] = symbol_data['volume'] / symbol_data['volume_mean_20']        symbol_data['volume_momentum'] = symbol_data['volume'].pct_change(5)        

## Target variables

In [ ]:
        symbol_data['target_return'] = symbol_data['daily_return'].shift(-1)  # Next day return        symbol_data['target_direction'] = (symbol_data['target_return'] > 0).astype(int)  # Up/Down        symbol_data['target_volatility'] = symbol_data['daily_return'].rolling(5).std().shift(-1)  # Future volatility                processed_data.append(symbol_data)        df = pd.concat(processed_data, ignore_index=True)    df = df.dropna()        print(f"✅ Time series dataset prepared: {df.shape}")    return df

In [ ]:
def calculate_rsi(prices, window=14):    """Calculate RSI indicator"""    delta = prices.diff()    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()    rs = gain / loss    return 100 - (100 / (1 + rs))

In [ ]:
def calculate_macd(prices, fast=12, slow=26, signal=9):    """Calculate MACD indicator"""    ema_fast = prices.ewm(span=fast).mean()    ema_slow = prices.ewm(span=slow).mean()    macd = ema_fast - ema_slow    macd_signal = macd.ewm(span=signal).mean()    return macd, macd_signal

In [ ]:
def calculate_bollinger_position(prices, window=20, num_std=2):    """Calculate position within Bollinger Bands"""    sma = prices.rolling(window).mean()    std = prices.rolling(window).std()    upper_band = sma + (std * num_std)    lower_band = sma - (std * num_std)    return (prices - lower_band) / (upper_band - lower_band)

In [ ]:
def prepare_time_series_features(df):    """Prepare feature sets for time series modeling"""    print("\n🎯 Preparing Time Series Features")    

## Define feature categories

In [ ]:
    lag_features = [col for col in df.columns if '_lag_' in col]    rolling_features = [col for col in df.columns if any(x in col for x in ['_mean_', '_std_'])]    technical_features = ['rsi', 'macd', 'macd_signal', 'bb_position']    volume_features = ['volume_ratio', 'volume_momentum']    base_features = ['daily_return', 'log_return', 'price_range']    

## Combine all features

In [ ]:
    all_features = lag_features + rolling_features + technical_features + volume_features + base_features    

## Remove features with too many missing values

In [ ]:
    feature_completeness = df[all_features].isnull().mean()    valid_features = feature_completeness[feature_completeness < 0.1].index.tolist()        print(f"Total potential features: {len(all_features)}")    print(f"Valid features (>90% complete): {len(valid_features)}")    print(f"Lag features: {len([f for f in valid_features if '_lag_' in f])}")    print(f"Rolling features: {len([f for f in valid_features if any(x in f for x in ['_mean_', '_std_'])])}")        return valid_features

In [ ]:
def time_series_split_data(df, features, test_size=0.2):    """Perform time-aware data splitting"""    print("\n📅 Time Series Data Splitting")    

## Sort by date to maintain temporal order

In [ ]:
    df_sorted = df.sort_values(['date']).copy()    

## Calculate split point

In [ ]:
    split_date = df_sorted['date'].quantile(1 - test_size)        train_mask = df_sorted['date'] <= split_date    test_mask = df_sorted['date'] > split_date    

## Prepare feature matrices

In [ ]:
    X_train = df_sorted[train_mask][features]    X_test = df_sorted[test_mask][features]    

## Prepare targets

In [ ]:
    y_reg_train = df_sorted[train_mask]['target_return']    y_reg_test = df_sorted[test_mask]['target_return']    y_clf_train = df_sorted[train_mask]['target_direction']    y_clf_test = df_sorted[test_mask]['target_direction']        print(f"Train period: {df_sorted[train_mask]['date'].min()} to {df_sorted[train_mask]['date'].max()}")    print(f"Test period: {df_sorted[test_mask]['date'].min()} to {df_sorted[test_mask]['date'].max()}")    print(f"Train samples: {len(X_train)}")    print(f"Test samples: {len(X_test)}")    print(f"Classification target distribution: {y_clf_train.value_counts().to_dict()}")        return X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test

In [ ]:
def build_time_series_pipelines():    """Build sklearn pipelines for automated modeling"""    print("\n🔧 Building Time Series Pipelines")    

## Regression pipelines

In [ ]:
    regression_pipelines = {        'Linear_TS': Pipeline([            ('scaler', StandardScaler()),            ('regressor', LinearRegression())        ]),        'Ridge_TS': Pipeline([            ('scaler', StandardScaler()),            ('regressor', Ridge(alpha=1.0))        ]),        'RandomForest_TS': Pipeline([            ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))        ])    }    

## Classification pipelines

In [ ]:
    classification_pipelines = {        'Logistic_TS': Pipeline([            ('scaler', StandardScaler()),            ('classifier', LogisticRegression(random_state=42, max_iter=1000))        ]),        'RandomForest_CLF': Pipeline([            ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))        ])    }        print(f"Created {len(regression_pipelines)} regression pipelines")    print(f"Created {len(classification_pipelines)} classification pipelines")        return regression_pipelines, classification_pipelines

In [ ]:
def evaluate_regression_models(pipelines, X_train, X_test, y_train, y_test):    """Evaluate time series regression models"""    print("\n📈 Evaluating Time Series Regression Models")        regression_results = {}        for name, pipeline in pipelines.items():        print(f"\nTraining {name}...")        

## Fit pipeline

In [ ]:
        pipeline.fit(X_train, y_train)        

## Predictions

In [ ]:
        y_train_pred = pipeline.predict(X_train)        y_test_pred = pipeline.predict(X_test)        

## Metrics

In [ ]:
        train_r2 = r2_score(y_train, y_train_pred)        test_r2 = r2_score(y_test, y_test_pred)        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))                regression_results[name] = {            'pipeline': pipeline,            'train_r2': train_r2,            'test_r2': test_r2,            'train_rmse': train_rmse,            'test_rmse': test_rmse,            'y_test_pred': y_test_pred        }                print(f"  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}")        print(f"  Train RMSE: {train_rmse:.6f}, Test RMSE: {test_rmse:.6f}")        return regression_results

In [ ]:
def evaluate_classification_models(pipelines, X_train, X_test, y_train, y_test):    """Evaluate time series classification models"""    print("\n🎯 Evaluating Time Series Classification Models")        classification_results = {}        for name, pipeline in pipelines.items():        print(f"\nTraining {name}...")        

## Fit pipeline

In [ ]:
        pipeline.fit(X_train, y_train)        

## Predictions

In [ ]:
        y_train_pred = pipeline.predict(X_train)        y_test_pred = pipeline.predict(X_test)        

## Probabilities (if available)

In [ ]:
        if hasattr(pipeline.named_steps[list(pipeline.named_steps.keys())[-1]], 'predict_proba'):            y_test_proba = pipeline.predict_proba(X_test)[:, 1]        else:            y_test_proba = None        

## Metrics

In [ ]:
        train_acc = accuracy_score(y_train, y_train_pred)        test_acc = accuracy_score(y_test, y_test_pred)                if y_test_proba is not None:            test_auc = roc_auc_score(y_test, y_test_proba)        else:            test_auc = None                classification_results[name] = {            'pipeline': pipeline,            'train_acc': train_acc,            'test_acc': test_acc,            'test_auc': test_auc,            'y_test_pred': y_test_pred,            'y_test_proba': y_test_proba        }                print(f"  Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")        if test_auc:            print(f"  Test AUC: {test_auc:.4f}")        return classification_results

In [ ]:
def time_series_cross_validation(pipelines, X, y, task_type='regression'):    """Perform time series cross-validation"""    print(f"\n🔄 Time Series Cross-Validation ({task_type})")    

## Time series cross-validation

In [ ]:
    tscv = TimeSeriesSplit(n_splits=5)    cv_results = {}        for name, pipeline in pipelines.items():        print(f"CV for {name}...")                if task_type == 'regression':            cv_scores = cross_val_score(pipeline, X, y, cv=tscv, scoring='r2')        else:  # classification            cv_scores = cross_val_score(pipeline, X, y, cv=tscv, scoring='accuracy')                cv_results[name] = {            'mean_score': cv_scores.mean(),            'std_score': cv_scores.std(),            'scores': cv_scores        }                print(f"  CV Score: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")        return cv_results

In [ ]:
def visualize_results(regression_results, classification_results, X_test, y_reg_test, y_clf_test):    """Visualize model results"""    print("\n📊 Visualizing Results")        fig, axes = plt.subplots(2, 3, figsize=(18, 12))    

## Regression results

In [ ]:
    best_reg_name = max(regression_results.keys(), key=lambda x: regression_results[x]['test_r2'])    best_reg = regression_results[best_reg_name]    

## 1. Regression: Actual vs Predicted

In [ ]:
    axes[0,0].scatter(y_reg_test, best_reg['y_test_pred'], alpha=0.6, s=20)    axes[0,0].plot([y_reg_test.min(), y_reg_test.max()], [y_reg_test.min(), y_reg_test.max()], 'r--', lw=2)    axes[0,0].set_xlabel('Actual Returns')    axes[0,0].set_ylabel('Predicted Returns')    axes[0,0].set_title(f'Regression: {best_reg_name}\nR² = {best_reg["test_r2"]:.4f}')    

## 2. Regression: Residuals

In [ ]:
    residuals = y_reg_test - best_reg['y_test_pred']    axes[0,1].scatter(best_reg['y_test_pred'], residuals, alpha=0.6, s=20)    axes[0,1].axhline(y=0, color='red', linestyle='--')    axes[0,1].set_xlabel('Predicted Returns')    axes[0,1].set_ylabel('Residuals')    axes[0,1].set_title('Regression Residuals')    

## 3. Regression Model Comparison

In [ ]:
    reg_names = list(regression_results.keys())    reg_scores = [regression_results[name]['test_r2'] for name in reg_names]    axes[0,2].bar(reg_names, reg_scores, alpha=0.7)    axes[0,2].set_ylabel('Test R²')    axes[0,2].set_title('Regression Model Comparison')    axes[0,2].tick_params(axis='x', rotation=45)    

## Classification results

In [ ]:
    best_clf_name = max(classification_results.keys(), key=lambda x: classification_results[x]['test_acc'])    best_clf = classification_results[best_clf_name]    

## 4. Classification: Confusion Matrix

In [ ]:
    cm = confusion_matrix(y_clf_test, best_clf['y_test_pred'])    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1,0])    axes[1,0].set_xlabel('Predicted')    axes[1,0].set_ylabel('Actual')    axes[1,0].set_title(f'Classification: {best_clf_name}\nAcc = {best_clf["test_acc"]:.4f}')    

## 5. Classification: Probability Distribution

In [ ]:
    if best_clf['y_test_proba'] is not None:        axes[1,1].hist(best_clf['y_test_proba'][y_clf_test == 0], alpha=0.7, label='Down', bins=20)        axes[1,1].hist(best_clf['y_test_proba'][y_clf_test == 1], alpha=0.7, label='Up', bins=20)        axes[1,1].set_xlabel('Predicted Probability (Up)')        axes[1,1].set_ylabel('Frequency')        axes[1,1].set_title('Probability Distribution')        axes[1,1].legend()    

## 6. Classification Model Comparison

In [ ]:
    clf_names = list(classification_results.keys())    clf_scores = [classification_results[name]['test_acc'] for name in clf_names]    axes[1,2].bar(clf_names, clf_scores, alpha=0.7)    axes[1,2].set_ylabel('Test Accuracy')    axes[1,2].set_title('Classification Model Comparison')    axes[1,2].tick_params(axis='x', rotation=45)        plt.tight_layout()    plt.show()

In [ ]:
def feature_importance_analysis(regression_results, classification_results, features):    """Analyze feature importance from tree-based models"""    print("\n🎯 Feature Importance Analysis")    

## Find Random Forest models

In [ ]:
    rf_reg = None    rf_clf = None        for name, result in regression_results.items():        if 'RandomForest' in name:            rf_reg = result['pipeline'].named_steps['regressor']            break        for name, result in classification_results.items():        if 'RandomForest' in name:            rf_clf = result['pipeline'].named_steps['classifier']            break        fig, axes = plt.subplots(1, 2, figsize=(15, 6))    

## Regression feature importance

In [ ]:
    if rf_reg:        reg_importance = pd.DataFrame({            'feature': features,            'importance': rf_reg.feature_importances_        }).sort_values('importance', ascending=False).head(15)                axes[0].barh(reg_importance['feature'], reg_importance['importance'])        axes[0].set_xlabel('Feature Importance')        axes[0].set_title('Regression: Top 15 Features')    

## Classification feature importance

In [ ]:
    if rf_clf:        clf_importance = pd.DataFrame({            'feature': features,            'importance': rf_clf.feature_importances_        }).sort_values('importance', ascending=False).head(15)                axes[1].barh(clf_importance['feature'], clf_importance['importance'])        axes[1].set_xlabel('Feature Importance')        axes[1].set_title('Classification: Top 15 Features')        plt.tight_layout()    plt.show()        return reg_importance if rf_reg else None, clf_importance if rf_clf else None

In [ ]:
def main():    """Main execution function"""

## Load and prepare data

In [ ]:
    df = load_and_prepare_data()    if df is None:        return    

## Prepare features

In [ ]:
    features = prepare_time_series_features(df)    

## Time series split

In [ ]:
    X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test = time_series_split_data(df, features)    

## Build pipelines

In [ ]:
    reg_pipelines, clf_pipelines = build_time_series_pipelines()    

## Evaluate models

In [ ]:
    regression_results = evaluate_regression_models(reg_pipelines, X_train, X_test, y_reg_train, y_reg_test)    classification_results = evaluate_classification_models(clf_pipelines, X_train, X_test, y_clf_train, y_clf_test)    

## Cross-validation

In [ ]:
    reg_cv_results = time_series_cross_validation(reg_pipelines, X_train, y_reg_train, 'regression')    clf_cv_results = time_series_cross_validation(clf_pipelines, X_train, y_clf_train, 'classification')    

## Visualizations

In [ ]:
    visualize_results(regression_results, classification_results, X_test, y_reg_test, y_clf_test)    

## Feature importance

In [ ]:
    reg_importance, clf_importance = feature_importance_analysis(regression_results, classification_results, features)    

## Save results

In [ ]:
    results_summary = {        'regression_models': {name: {'test_r2': result['test_r2'], 'test_rmse': result['test_rmse']}                             for name, result in regression_results.items()},        'classification_models': {name: {'test_acc': result['test_acc'], 'test_auc': result['test_auc']}                                 for name, result in classification_results.items()},        'features_used': len(features),        'train_samples': len(X_train),        'test_samples': len(X_test)    }        print(f"\n💾 Results Summary:")    print(f"Best Regression Model: {max(regression_results.keys(), key=lambda x: regression_results[x]['test_r2'])}")    print(f"Best Classification Model: {max(classification_results.keys(), key=lambda x: classification_results[x]['test_acc'])}")        print("\n✅ Stage 10b: Time Series/Classification Modeling Complete")    print("Key deliverables:")    print("- Automated sklearn pipelines for time series modeling")    print("- Lag and rolling features for temporal dependencies")    print("- Both regression and classification approaches")    print("- Time series cross-validation")    print("- Feature importance analysis")    print("- Comprehensive model evaluation and comparison")if __name__ == "__main__":    main()